In [0]:
!pwd

In [0]:
!ls

In [0]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir('/content/drive/My Drive/scholarly_paper_recommendationg_datasets/Candidate papers to recommend')

In [0]:
!pip install pdfminer.six
!pip install numba

     |████████████████████████████████| 5.6MB 6.0MB/s 
     |████████████████████████████████| 9.7MB 40.9MB/s 


In [0]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os
import pandas as pd
from numba import jit

@jit
def convert_pdf_to_txt(path_to_file):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path_to_file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [0]:
# 遍历文件夹返回所有叶节点处的PDF路径list
@jit
def iter_files(rootDir):
    file_name_list = []
    for root,dirs,files in os.walk(rootDir):
        for file in files:
            file_name = os.path.join(root,file)
            file_name_list.append(file_name)
    return file_name_list

# 调用函数iter_files()获取所有的PDF路径，返回一个list
pdfs = iter_files('/content/drive/My Drive/scholarly_paper_recommendationg_datasets/Candidate papers to recommend/')
len(pdfs) # 查看PDF个数

95238

In [0]:
# 调用函数将PDF路径存储到一个TXT中
f = open("/content/drive/My Drive/MyProjects/pdflist_subset1_7368.txt", "w+") 
for pdf in pdfs:
    f.write(pdf)
    f.write('\n')
f.close()

In [0]:
# 遍历pdfs中的每一个PDF路径，调用函数convert_pdf_to_txt()将PDF转换成无格式文本，并存储到一个新的pdf_content_dict中
import datetime
starttime = datetime.datetime.now()
pdf_content_dict ={}
fail_list = []
i = 1
for pdf in pdfs:
    try:
        pdf_content_dict[pdf] = convert_pdf_to_txt(pdf)
        if i%10 == 0:
          print('the {}th pdf is processing'.format(i))
        i += 1
    except:
        fail_list.append(pdf)
    
# 将获取的论文字典全文内容字典表写入TXT文件中(将dictionary写入txt文件)
content = open("/content/drive/My Drive/MyProjects/pdf_subset1_7368.txt", "w+")
content.write(str(pdf_content_dict))
content.close()

# 将的PD解析失败的路径列表写入TXT文件中(将list写入txt文件)
fail2convert = open("/content/drive/My Drive/MyProjects/fail_list_subset1_7368.txt", "w+")
for pdf in fail_list:
    fail2convert.write(pdf)
    fail2convert.write('\n')
fail2convert.close()
endtime = datetime.datetime.now()
print(starttime)
print(endtime)
print(endtime - starttime)

In [0]:
# 将字典转换成pandas的dataframe，便于分析,并导出为CSV
import pandas as pd
df = pd.DataFrame.from_dict(pdf_content_dict, orient='index').reset_index()
df.columns = ["path", "content"]
df.to_csv('/content/drive/My Drive/MyProjects/paper_subset1_7368.csv')